In [1]:
from tensorflow.keras import layers, models, applications
from tensorflow.keras.utils import plot_model

def TableNet_Ringkas(input_shape=(1824, 1760, 3)):
    inputs = layers.Input(shape=input_shape, name="Input_Image")

    # Encoder using ResNet50
    base_model = applications.ResNet50(
        include_top=False, weights="imagenet", input_tensor=inputs, name="ResNet50_Encoder"
    )

    # Extract bottleneck output
    bottleneck = base_model.get_layer("conv5_block3_out").output  # (57, 55, 2048)

    # Decoder for Table Mask
    def decoder_block(x, name):
        x = layers.Conv2DTranspose(256, (2, 2), strides=2, padding="same", name=f"{name}_Upsample_1")(x)
        x = layers.Conv2D(128, (3, 3), activation="relu", padding="same", name=f"{name}_Conv_1")(x)
        x = layers.Conv2DTranspose(64, (2, 2), strides=2, padding="same", name=f"{name}_Upsample_2")(x)
        x = layers.Conv2D(32, (3, 3), activation="relu", padding="same", name=f"{name}_Conv_2")(x)
        return x

    table_decoder = decoder_block(bottleneck, "TableDecoder")
    column_decoder = decoder_block(bottleneck, "ColumnDecoder")

    # Output layers
    table_output = layers.Conv2D(1, (1, 1), activation="sigmoid", name="Table_Output")(table_decoder)
    column_output = layers.Conv2D(1, (1, 1), activation="sigmoid", name="Column_Output")(column_decoder)

    # Model
    model = models.Model(inputs=inputs, outputs=[table_output, column_output], name="TableNet_Ringkas")

    return model

# Create the model
model = TableNet_Ringkas()

# Visualize the model
plot_model(
    model,
    to_file="TableNet_Ringkas.png",
    show_shapes=True,        # Show input/output tensor shapes
    show_layer_names=True,   # Display layer names
    rankdir="TB",            # Top-to-bottom layout
    dpi=200                  # High resolution for better readability
)


In [1]:
import visualkeras
from tensorflow import keras

# Define the model
model = keras.models.Sequential()

model.add(keras.layers.Flatten(input_shape=(28, 28, 1), name="Input_Flatten"))
model.add(keras.layers.Dense(128, activation='relu', name="Dense_128"))
model.add(keras.layers.Dense(64, activation='relu', name="Dense_64_1"))
model.add(keras.layers.Dropout(0.5, name="Dropout_50%"))
model.add(keras.layers.Dense(64, activation='relu', name="Dense_64_2"))
model.add(keras.layers.Dropout(0.3, name="Dropout_30%"))
model.add(keras.layers.Dense(32, activation='relu', name="Dense_32"))
model.add(keras.layers.Dropout(0.2, name="Dropout_20%"))
model.add(keras.layers.Dense(16, activation='relu', name="Dense_16"))
model.add(keras.layers.Dense(10, activation='softmax', name="Output_Softmax"))

# Visualize the model
visualkeras.layered_view(
    model, 
    legend=True,            # Add a legend for layer types
    draw_volume=True,       # Show volumes of each layer
    scale_xy=2,             # Adjust scale for better proportions
    scale_z=1,              # Adjust depth for better clarity
    spacing=20              # Space between layers
).show()


d:\Software\Python\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
d:\Software\Python\Lib\site-packages\visualkeras\layered.py:86: UserWarning: The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.
  warnings.warn("The legend_text_spacing_offset parameter is deprecated and will be removed in a future release.")


In [2]:
from graphviz import Digraph

# Define a directed graph
dot = Digraph(format='png')
dot.attr(rankdir='LR')  # Left-to-right layout

# Add nodes for each layer
dot.node("Input", "Input\n(28x28x1)", shape="circle")
dot.node("Dense_1", "Dense 128\nReLU", shape="circle")
dot.node("Dense_2", "Dense 64\nReLU", shape="circle")
dot.node("Dropout_1", "Dropout 50%", shape="circle")
dot.node("Dense_3", "Dense 64\nReLU", shape="circle")
dot.node("Dropout_2", "Dropout 30%", shape="circle")
dot.node("Dense_4", "Dense 32\nReLU", shape="circle")
dot.node("Dropout_3", "Dropout 20%", shape="circle")
dot.node("Dense_5", "Dense 16\nReLU", shape="circle")
dot.node("Output", "Output\nSoftmax", shape="circle")

# Add edges between nodes
dot.edges([
    ("Input", "Dense_1"),
    ("Dense_1", "Dense_2"),
    ("Dense_2", "Dropout_1"),
    ("Dropout_1", "Dense_3"),
    ("Dense_3", "Dropout_2"),
    ("Dropout_2", "Dense_4"),
    ("Dense_4", "Dropout_3"),
    ("Dropout_3", "Dense_5"),
    ("Dense_5", "Output")
])

# Render and display the graph
dot.render("Sequential_Model_Lingkaran", view=True)


'Sequential_Model_Lingkaran.png'